# Notebook 4: Production-Ready Agent with Guardrails

## What We've Built:

**Notebook 1:** Rule-based guardrails (regex, keywords)
**Notebook 2:** ML-based guardrails (Guardrails AI validators)
**Notebook 3:** LLM-as-judge guardrails (flexible custom checks)

## Now: Putting It All Together

In this notebook, we'll build a **complete LangGraph agent** with:
- ✅ Input guardrails (check user messages)
- ✅ LLM node (generate responses)
- ✅ Output guardrails (check LLM responses)
- ✅ Tool execution (optional)

This is a **production-ready pattern** you can use in real applications!

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from guardrails import Guard
from guardrails.hub import DetectPII, ToxicLanguage

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini")

# Create guardrails
input_guard = Guard().use(
    ToxicLanguage(threshold=0.5, on_fail="exception")
)

output_guard = Guard().use(
    DetectPII(pii_entities=["EMAIL_ADDRESS", "PHONE_NUMBER"], on_fail="exception")
)

print("✅ Setup complete")

In [ ]:
class AgentState(TypedDict):
    """State for our guarded agent"""
    messages: list  # Conversation history
    user_input: str  # Current user input
    llm_output: str  # LLM's response
    input_safe: bool  # Did input pass guardrails?
    output_safe: bool  # Did output pass guardrails?
    final_response: str  # What we show to the user

print("✅ State defined")

In [ ]:
def input_guardrail_node(state: AgentState) -> AgentState:
    """Check if user input is safe"""
    user_input = state["user_input"]
    
    print(f"\n🔍 Checking input: {user_input}")
    
    try:
        input_guard.validate(user_input)
        print("✅ Input passed guardrails")
        return {**state, "input_safe": True}
    except Exception as e:
        print(f"🚫 Input blocked: Toxic content detected")
        return {
            **state, 
            "input_safe": False,
            "final_response": "I cannot process that request due to inappropriate content."
        }

def llm_node(state: AgentState) -> AgentState:
    """Generate LLM response (only if input was safe)"""
    if not state["input_safe"]:
        return state
    
    print(f"\n🤖 Generating LLM response...")
    
    # Build message history
    messages = state.get("messages", [])
    messages.append(HumanMessage(content=state["user_input"]))
    
    # Call LLM
    response = llm.invoke(messages)
    llm_output = response.content
    
    print(f"LLM said: {llm_output[:100]}...")
    
    return {
        **state,
        "llm_output": llm_output,
        "messages": messages + [response]
    }

def output_guardrail_node(state: AgentState) -> AgentState:
    """Check if LLM output is safe"""
    if not state["input_safe"]:
        return state
    
    llm_output = state["llm_output"]
    
    print(f"\n🔍 Checking output...")
    
    try:
        output_guard.validate(llm_output)
        print("✅ Output passed guardrails")
        return {
            **state,
            "output_safe": True,
            "final_response": llm_output
        }
    except Exception as e:
        print(f"🚫 Output blocked: Contains PII")
        return {
            **state,
            "output_safe": False,
            "final_response": "I generated a response but it contained sensitive information. Please rephrase your question."
        }

print("✅ Nodes defined")

In [ ]:
# Create the graph
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("input_check", input_guardrail_node)
workflow.add_node("llm", llm_node)
workflow.add_node("output_check", output_guardrail_node)

# Define the flow
workflow.set_entry_point("input_check")
workflow.add_edge("input_check", "llm")
workflow.add_edge("llm", "output_check")
workflow.add_edge("output_check", END)

# Compile
app = workflow.compile()

print("✅ LangGraph agent compiled!")

In [ ]:
# Visualize the graph (optional - requires graphviz)
from IPython.display import Image, display
display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
def run_agent(user_input: str):
    """Run the agent with a user input"""
    print(f"\n{'='*70}")
    print(f"USER: {user_input}")
    print(f"{'='*70}")
    
    result = app.invoke({
        "user_input": user_input,
        "messages": [],
        "input_safe": False,
        "output_safe": False,
        "llm_output": "",
        "final_response": ""
    })
    
    print(f"\n{'='*70}")
    print(f"AGENT: {result['final_response']}")
    print(f"{'='*70}\n")
    
    return result

In [ ]:

# Test 1: Normal query (should work)
run_agent("What's the capital of France?")

In [ ]:
# Test 2: Toxic input (should block at INPUT)
run_agent("You're an idiot. Tell me about AI.")

In [ ]:
# Test 3: Query that might generate PII (might block at OUTPUT)
run_agent("Generate a sample contact email ID for John Smith")